In [14]:
# import the required library
import warnings
import pandas as pd
from datetime import timedelta

In [15]:
# Load the bank transaction dataset
# specify the path where your dataset is
df = pd.read_csv('D:/rawdata/tbl_bank_transactions.csv',dtype={'account_number':str,'transaction_amount':float}) 
df.head(3)

,first_name,last_name,email,gender,ip_address,transaction_id,transaction_date_time,account_number,transaction_type,transaction_amount,transaction_currency,merchant_name,merchant_location,transaction_category
0,Tomi,Somefun,Tomi.Somefun-Ogbu@test.com,Female,144.23.8.167,897,2021-04-23 15:07:22,0011168812,transfer,16316.78,IDR,Oyoyo,Room 193,transportation
1,Tomi,Somefun,Tomi.Somefun-Ogbu@test.com,Female,31.188.31.232,898,2021-04-18 16:22:53,0011168812,withdrawal,7362.59,SEK,Yambee,Suite 61,entertainment
2,Tomi,Somefun,Tomi.Somefun-Ogbu@test.com,Female,84.29.217.100,899,2021-12-27 19:52:24,0011168812,withdrawal,29449.95,CNY,Feedbug,Room 939,transportation


In [16]:
# confirm how many records exist and the number of rows and columns
df.shape

(20000, 14)

In [17]:
# we will be working with account_number, transaction_date_time, and transaction_amount
df = df[['account_number','transaction_date_time', 'transaction_amount', 'transaction_type']]
#peek into the data
df.head(5)

,account_number,transaction_date_time,transaction_amount,transaction_type
0,0011168812,2021-04-23 15:07:22,16316.78,transfer
1,0011168812,2021-04-18 16:22:53,7362.59,withdrawal
2,0011168812,2021-12-27 19:52:24,29449.95,withdrawal
3,0011168812,2021-07-14 08:19:50,1918.73,deposit
4,0011168812,2021-04-19 09:47:06,23768.32,withdrawal


In [18]:
#lets convert the transaction_date_time from object to a proper date time
df['transaction_date_time']=pd.to_datetime(df['transaction_date_time'])
df

,account_number,transaction_date_time,transaction_amount,transaction_type
0,0011168812,2021-04-23 15:07:22,16316.78,transfer
1,0011168812,2021-04-18 16:22:53,7362.59,withdrawal
2,0011168812,2021-12-27 19:52:24,29449.95,withdrawal
3,0011168812,2021-07-14 08:19:50,1918.73,deposit
4,0011168812,2021-04-19 09:47:06,23768.32,withdrawal
...,...,...,...,...
19995,0011168820,2021-12-06 15:33:28,11476.22,withdrawal
19996,0011168820,2021-06-08 15:19:39,6522.31,withdrawal
19997,0011168820,2021-01-07 06:19:25,7793.63,transfer
19998,0011168820,2021-07-12 05:58:19,2453.61,transfer


In [19]:
# confirm the data types
df.dtypes

account_number                   object
transaction_date_time    datetime64[ns]
transaction_amount              float64
transaction_type                 object
dtype: object

In [20]:
# Let us define a pattern to use for our fraud detection. 
# create a new column called hour and extract hourly value from the transaction_date_time
df['hour']=df.transaction_date_time.dt.hour
# create a new column called weekday and extract the weekday value from transaction_date_time
df['weekday']=pd.Categorical(df.transaction_date_time.dt.strftime('%A'), categories=['Monday','Tuesday','Wednesday',
                                                                                     'Thursday','Friday','Saturday', 
                                                                                     'Sunday'], ordered=True)
df


,account_number,transaction_date_time,transaction_amount,transaction_type,hour,weekday
0,0011168812,2021-04-23 15:07:22,16316.78,transfer,15,Friday
1,0011168812,2021-04-18 16:22:53,7362.59,withdrawal,16,Sunday
2,0011168812,2021-12-27 19:52:24,29449.95,withdrawal,19,Monday
3,0011168812,2021-07-14 08:19:50,1918.73,deposit,8,Wednesday
4,0011168812,2021-04-19 09:47:06,23768.32,withdrawal,9,Monday
...,...,...,...,...,...,...
19995,0011168820,2021-12-06 15:33:28,11476.22,withdrawal,15,Monday
19996,0011168820,2021-06-08 15:19:39,6522.31,withdrawal,15,Tuesday
19997,0011168820,2021-01-07 06:19:25,7793.63,transfer,6,Thursday
19998,0011168820,2021-07-12 05:58:19,2453.61,transfer,5,Monday


In [22]:
df.dtypes

account_number                   object
transaction_date_time    datetime64[ns]
transaction_amount              float64
transaction_type                 object
hour                              int64
weekday                        category
dtype: object

In [23]:
# check for NAN in the amount 
print(df.isna().sum())

account_number           0
transaction_date_time    0
transaction_amount       0
transaction_type         0
hour                     0
weekday                  0
dtype: int64


In [25]:
# function to generate each customers transaction pattern by account
# and write into folders
def gen_txn_pattern_per_acct(df):
    # get the unique transaction accounts
    # this will end up getting 20 unique accounts
    # since our dataset contains just 20 customers account with transactions
    df_unqiue_accts = df.account_number.unique()
    # get the path to write data.  This is the path where the folders data.1, data.2, data.3 were created.
    writedata = "D:\\curbingfraud\\datasets\\"
    # set the txn pattern number of days for pattern generation per account
    # as approved by management
    #num_of_days_txn_pattern = "30"
    # form the dataframe name variable to be used
    # txndays = "sum_"+num_of_days_txn_pattern+"days" # this will become sum_30days
    cnt = 0
    datafolder = ''
    total = 0
    for i in df_unqiue_accts:
        total += 1
        cnt += 1
        # pick the account to start generating txn pattern where transaction_type = withdrawal
        df_per_acct = df.query('account_number == @i & transaction_type =="withdrawal"')
        # sort the value by input date and set the index to input date
        # the set_index() method allows one or more column values become the row index.
        # Note: I used the \ to move the rest of the code to a new line.
        df_txn_pattern = df_per_acct.sort_values(by=['account_number', 'transaction_date_time']).\
        set_index('transaction_date_time')
        
        # get the sum of the previous n days transaction amount based on customers account
        # Note: I used the \ to move the rest of the code to a new line.
        # df_txn_pattern[txndays] = df_txn_pattern.groupby('account_number')['transaction_amount'].\
        # transform(lambda s: s.rolling(timedelta(days=int(num_of_days_txn_pattern))).sum())
        # assign the value into new dataframe named final_df
        final_df = df_txn_pattern
        # check if the cnt is 3 then reset cnt 1
        # cnt ==3 means it has gotten to the last folder
        # and there is a need to start writing into folder 1 and proceed to the next folder
        # until we are done splitting.
        if cnt == 1:
            datafolder = "data.1"
            print(f"Writing data for {i} into folder path {datafolder}. item {str(total)} of ==> {str(len(df_unqiue_accts))}")
        elif cnt == 2:
            datafolder = "data.2"
            print(f"Writing data for {i} into folder path {datafolder}. item {str(total)} of ==> {str(len(df_unqiue_accts))}")
        elif cnt == 3:
            datafolder = "data.3"
            print(f"Writing data for {i} into folder path {datafolder}. item {str(total)} of ==> {str(len(df_unqiue_accts))}")
            cnt = 0
        final_df.to_csv(writedata + datafolder + "\\" + i + '.csv', index = False)

    return total 
#*******************************************************

In [26]:
# generate txn pattern file per account
val = gen_txn_pattern_per_acct(df)
print(f"Data processing completed... with total item {str(val)}")

Writing data for 0011168812 into folder path data.1. item 1 of ==> 20
Writing data for 0011168813 into folder path data.2. item 2 of ==> 20
Writing data for 0011118881 into folder path data.3. item 3 of ==> 20
Writing data for 0011128882 into folder path data.1. item 4 of ==> 20
Writing data for 0011138883 into folder path data.2. item 5 of ==> 20
Writing data for 0011148884 into folder path data.3. item 6 of ==> 20
Writing data for 0011158885 into folder path data.1. item 7 of ==> 20
Writing data for 0011168886 into folder path data.2. item 8 of ==> 20
Writing data for 0011168887 into folder path data.3. item 9 of ==> 20
Writing data for 0011168888 into folder path data.1. item 10 of ==> 20
Writing data for 0011168889 into folder path data.2. item 11 of ==> 20
Writing data for 0011168810 into folder path data.3. item 12 of ==> 20
Writing data for 0011168811 into folder path data.1. item 13 of ==> 20
Writing data for 0011168815 into folder path data.2. item 14 of ==> 20
Writing data fo